## Import DOCX file

In [3]:
from docx import Document

In [8]:
document = Document(r"../res/test-swiss.docx")
document

In [15]:
for table in document.tables:
    for row in table.rows:
        for cell in row.cells:
            print(cell.text)

COUNTRY: SWITZERLAND
PROSPECTS
PROSPECTS
PROSPECTS
PROSPECTS
PROSPECTS
PROSPECTS
GROUP 
ACTIVITY
CLIENT TYPE
INTERACTION MODE
OFFSHORE SCENARIO
ONSHORE SCENARIO
Social Contact 

Social interactions

Retail
Pro-actively
YES
YES
Social Contact 

Social interactions

Retail
Upon RS
YES       
YES
Social Contact 

Social interactions

Professionals

Pro-actively
YES
YES
Social Contact 

Social interactions

Professionals

Upon RS
YES
YES
Social Contact 

Provide contact details
Retail
Pro-actively 
YES
YES
Social Contact 

Provide contact details
Retail
Upon RS 
YES
YES
Social Contact 

Provide contact details
Professionals

Pro-actively 
YES
YES
Social Contact 

Provide contact details
Professionals

Upon RS 
YES
YES
EXISTING CLIENTS
EXISTING CLIENTS
EXISTING CLIENTS
EXISTING CLIENTS
EXISTING CLIENTS
EXISTING CLIENTS
GROUP 
ACTIVITY
CLIENT TYPE
INTERACTION MODE
OFFSHORE SCENARIO
ONSHORE SCENARIO
Social Contact 

Social interactions

Retail
Pro-actively
YES
YES
Social Contact 

Social inte

Credit card agreements including related forms
Retail
Pro-actively
YES
NO
Preparatory administrative activities 

Provide forms and contractual documents
Credit card agreements including related forms
Retail
Upon RS
YES
NO
Preparatory administrative activities 

Provide forms and contractual documents
Credit card agreements including related forms
Professionals
Pro-actively
YES
NO
Preparatory administrative activities 

Provide forms and contractual documents
Credit card agreements including related forms
Professionals
Upon RS
YES
NO
Preparatory administrative activities 

Provide forms and contractual documents
Other contracts and forms (securities lending, wealth management services and other types of contract)
Retail
Pro-actively
YES
NO
Preparatory administrative activities 

Provide forms and contractual documents
Other contracts and forms (securities lending, wealth management services and other types of contract)
Retail
Upon RS
YES
NO
Preparatory administrative activities 

Provi

Professionals
Upon RS
YES
NO
Preparatory administrative activities 

Explain account opening process and provide assistance for completion of documents and account opening settings
Obtain KYC information, including subsequent updates
Receive forms and contractual documents
Explain account opening process and provide assistance for completion of documents and account opening settings
Obtain KYC information, including subsequent updates
Receive forms and contractual documents
Retail
Pro-actively
YES
NO
Preparatory administrative activities 

Explain account opening process and provide assistance for completion of documents and account opening settings
Obtain KYC information, including subsequent updates
Receive forms and contractual documents
Explain account opening process and provide assistance for completion of documents and account opening settings
Obtain KYC information, including subsequent updates
Receive forms and contractual documents
Retail
Upon RS
YES
NO
Preparatory administra

Performance of activities 

Sector research and financial research 
with advisory mandate
with RSSI 

Retail
Upon RS 
YES
NO
Performance of activities 

Sector research and financial research 
with advisory mandate
with RSSI 

Professionals
Pro-actively 
YES
NO 
Performance of activities 

Sector research and financial research 
with advisory mandate
with RSSI 

Professionals
Upon RS 
YES
NO 
Performance of activities 

Sector research and financial research 
without advisory mandate
Retail
Pro-actively 
NO 
NO 
Performance of activities 

Sector research and financial research 
without advisory mandate
Retail
Upon RS 
YES
NO
Performance of activities 

Sector research and financial research 
without advisory mandate
Professionals
Pro-actively 
NO 
NO 
Performance of activities 

Sector research and financial research 
without advisory mandate
Professionals
Upon RS 
YES
YES
Performance of activities 

Macroeconomic research
with advisory mandate
with RSSI 

Retail
Pro-actively 
YES 
YE

## Make dataframes from tables

In [87]:
import pandas as pd
import io
import csv
from docx import Document

def read_docx_tables(filename, tab_id=None, **kwargs):
    """
    parse table(s) from a Word Document (.docx) into Pandas DataFrame(s)

    Parameters:
        filename:   file name of a Word Document

        tab_id:     parse a single table with the index: [tab_id] (counting from 0).
                    When [None] - return a list of DataFrames (parse all tables)

        kwargs:     arguments to pass to `pd.read_csv()` function

    Return: a single DataFrame if tab_id != None or a list of DataFrames otherwise
    """
    def read_docx_tab(tab, **kwargs):
        vf = io.StringIO()
        writer = csv.writer(vf)
        for row in tab.rows:
            writer.writerow(cell.text for cell in row.cells)
        vf.seek(0)
        return pd.read_csv(vf, **kwargs)

    doc = Document(filename)
    if tab_id is None:
        return [read_docx_tab(tab, **kwargs) for tab in doc.tables]
    else:
        try:
            return read_docx_tab(doc.tables[tab_id], **kwargs)
        except IndexError:
            print('Error: specified [tab_id]: {}  does not exist.'.format(tab_id))
            raise

In [178]:
dfs = read_docx_tables(r"../res/test-swiss.docx")

In [185]:
for idx, df in enumerate(dfs):
    print(f"{idx} >>> {df.columns}")

0 >>> Index(['CONNECTION TYPE', 'GROUP ', 'ACTIVITY', 'CLIENT TYPE',
       'INTERACTION MODE', 'SCENARIO', 'OUTCOME'],
      dtype='object')
1 >>> Index(['CONNECTION TYPE', 'GROUP ', 'ACTIVITY', 'CLIENT TYPE',
       'INTERACTION MODE', 'SCENARIO', 'OUTCOME'],
      dtype='object')
2 >>> Index(['CONNECTION TYPE', 'GROUP ', 'ACTIVITY', 'CLIENT TYPE',
       'INTERACTION MODE', 'SCENARIO', 'OUTCOME'],
      dtype='object')
3 >>> Index(['CONNECTION TYPE', 'GROUP ', 'ACTIVITY', 'CLIENT TYPE',
       'INTERACTION MODE', 'SCENARIO', 'OUTCOME'],
      dtype='object')
4 >>> Index(['CONNECTION TYPE', 'GROUP ', 'ACTIVITY', 'CLIENT TYPE',
       'INTERACTION MODE', 'SCENARIO', 'OUTCOME'],
      dtype='object')
5 >>> Index(['CONNECTION TYPE', 'GROUP ', 'ACTIVITY', 'CLIENT TYPE',
       'INTERACTION MODE', 'SCENARIO', 'OUTCOME'],
      dtype='object')
6 >>> Index(['CONNECTION TYPE', 'GROUP ', 'ACTIVITY', 'SUBACTIVITY ', 'CLIENT TYPE',
       'INTERACTION MODE', 'SCENARIO', 'OUTCOME'],
      dtype=

## Assemble in app

In [180]:
dfs = [df for df in dfs if df.shape != (0, 1)]

len(dfs)

11

In [181]:
for idx, df in enumerate(dfs):
    precondition = True if df.columns[0].startswith('PRE-CONDITION') else False
    columns = list(df.iloc[0,]) if not precondition else list(df.iloc[1,])
    connection_type = df.columns[0] if not precondition else df.iloc[0,0]

    df.columns = columns
    df.insert(0, "CONNECTION TYPE", connection_type)
    df.drop(df.head(1).index if not precondition else df.head(2).index, inplace=True)

In [182]:
for idx, df in enumerate(dfs):
    id_vars = [c for c in df.columns if not c.endswith('SCENARIO')]
    dfs[idx] = df.melt(id_vars=id_vars)
    dfs[idx].rename(columns={'value': 'OUTCOME', 'variable': 'SCENARIO'}, inplace=True)

In [187]:
dfs[0]

,CONNECTION TYPE,GROUP,ACTIVITY,CLIENT TYPE,INTERACTION MODE,SCENARIO,OUTCOME
0,PROSPECTS,Social Contact \n,Social interactions\n,Retail,Pro-actively,OFFSHORE SCENARIO,YES
1,PROSPECTS,Social Contact \n,Social interactions\n,Retail,Upon RS,OFFSHORE SCENARIO,YES
2,PROSPECTS,Social Contact \n,Social interactions\n,Professionals\n,Pro-actively,OFFSHORE SCENARIO,YES
3,PROSPECTS,Social Contact \n,Social interactions\n,Professionals\n,Upon RS,OFFSHORE SCENARIO,YES
4,PROSPECTS,Social Contact \n,Provide contact details,Retail,Pro-actively,OFFSHORE SCENARIO,YES
5,PROSPECTS,Social Contact \n,Provide contact details,Retail,Upon RS,OFFSHORE SCENARIO,YES
6,PROSPECTS,Social Contact \n,Provide contact details,Professionals\n,Pro-actively,OFFSHORE SCENARIO,YES
7,PROSPECTS,Social Contact \n,Provide contact details,Professionals\n,Upon RS,OFFSHORE SCENARIO,YES
8,PROSPECTS,Social Contact \n,Social interactions\n,Retail,Pro-actively,ONSHORE SCENARIO,YES
9,PROSPECTS,Social Contact \n,Social interactions\n,Retail,Upon RS,ONSHORE SCENARIO,YES


In [186]:
dfs[7]

,CONNECTION TYPE,GROUP,ACTIVITY,SUBACTIVITY,CLIENT TYPE,INTERACTION MODE,SCENARIO,OUTCOME
0,EXISTING CLIENTS,Preparatory administrative activities \n,Provide forms and contractual documents,Account opening contracts including related forms,Retail,Pro-actively,OFFSHORE SCENARIO,YES
1,EXISTING CLIENTS,Preparatory administrative activities \n,Provide forms and contractual documents,Account opening contracts including related forms,Retail,Upon RS,OFFSHORE SCENARIO,YES
2,EXISTING CLIENTS,Preparatory administrative activities \n,Provide forms and contractual documents,Account opening contracts including related forms,Professionals,Pro-actively,OFFSHORE SCENARIO,YES
3,EXISTING CLIENTS,Preparatory administrative activities \n,Provide forms and contractual documents,Account opening contracts including related forms,Professionals,Upon RS,OFFSHORE SCENARIO,YES
4,EXISTING CLIENTS,Preparatory administrative activities \n,Provide forms and contractual documents,Advisory and discretionary mandates including ...,Retail,Pro-actively,OFFSHORE SCENARIO,YES
...,...,...,...,...,...,...,...,...
59,EXISTING CLIENTS,Preparatory administrative activities \n,"Onboarding formality 1-3, including subsequent...","Onboarding formality 1-3, including subsequent...",Professionals,Upon RS,ONSHORE SCENARIO,NO
60,EXISTING CLIENTS,Preparatory administrative activities \n,"Onboarding formality 4, including subsequent u...","Onboarding formality 4, including subsequent u...",Retail,Pro-actively,ONSHORE SCENARIO,NO
61,EXISTING CLIENTS,Preparatory administrative activities \n,"Onboarding formality 4, including subsequent u...","Onboarding formality 4, including subsequent u...",Retail,Upon RS,ONSHORE SCENARIO,NO
62,EXISTING CLIENTS,Preparatory administrative activities \n,"Onboarding formality 4, including subsequent u...","Onboarding formality 4, including subsequent u...",Professionals,Pro-actively,ONSHORE SCENARIO,NO


In [188]:
pd.concat(dfs, sort=False)

,CONNECTION TYPE,GROUP,ACTIVITY,CLIENT TYPE,INTERACTION MODE,SCENARIO,OUTCOME,SUBACTIVITY,CONTRACTUAL SETUP
0,PROSPECTS,Social Contact \n,Social interactions\n,Retail,Pro-actively,OFFSHORE SCENARIO,YES,NaN,NaN
1,PROSPECTS,Social Contact \n,Social interactions\n,Retail,Upon RS,OFFSHORE SCENARIO,YES,NaN,NaN
2,PROSPECTS,Social Contact \n,Social interactions\n,Professionals\n,Pro-actively,OFFSHORE SCENARIO,YES,NaN,NaN
3,PROSPECTS,Social Contact \n,Social interactions\n,Professionals\n,Upon RS,OFFSHORE SCENARIO,YES,NaN,NaN
4,PROSPECTS,Social Contact \n,Provide contact details,Retail,Pro-actively,OFFSHORE SCENARIO,YES,NaN,NaN
...,...,...,...,...,...,...,...,...,...
11,EXISTING CLIENTS,Follow-up administrative tasks\n\n,Corporate actions\nAccount closure,Professionals\n,Upon RS,ONSHORE SCENARIO,YES,NaN,NaN
12,EXISTING CLIENTS,Follow-up administrative tasks\n\n,"Provide account statements, investment reports...",Retail,Pro-actively,ONSHORE SCENARIO,NO,NaN,NaN
13,EXISTING CLIENTS,Follow-up administrative tasks\n\n,"Provide account statements, investment reports...",Retail,Upon RS,ONSHORE SCENARIO,NO,NaN,NaN
14,EXISTING CLIENTS,Follow-up administrative tasks\n\n,"Provide account statements, investment reports...",Professionals\n,Pro-actively,ONSHORE SCENARIO,NO,NaN,NaN
